In [36]:
import os
import shutil
import matplotlib.pyplot as plt
from openpyxl.drawing.image import Image
import openpyxl as op
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook
import cv2
import time
import numpy as np
import pytesseract
import pandas as pd
import os
import math
from pyproj import CRS
from pyproj import Transformer
import keyboard as kb

In [39]:
# Задаем пути проекта

root_dir =r'G:\bulla'
file_dir = r'G:\bulla\one_2\source\work'


test_dir = r'G:\bulla\test_source1'
test_out_dir= r'G:\bulla\test_out_clear'

out_dir =r'G:\bulla\segm'

ade20k_dir =r'ade20k'
mrcnn_dir = r'mrcnn'
pascalvoc_dir =r'pascalvoc'

In [42]:
def adjust_gamma(image, gamma=1.2):
	# build a lookup table mapping the pixel values [0, 255] to
	# their adjusted gamma values
	invGamma = 1.0 / gamma
	table = np.array([((i / 255.0) ** invGamma) * 255
		for i in np.arange(0, 256)]).astype("uint8")
	# apply gamma correction using the lookup table
	return cv2.LUT(image, table)

def apply_mask(matrix, mask, fill_value):
    masked = np.ma.array(matrix, mask=mask, fill_value=fill_value)
    return masked.filled()

def apply_threshold(matrix, low_value, high_value):
    low_mask = matrix < low_value
    matrix = apply_mask(matrix, low_mask, low_value)
    high_mask = matrix > high_value
    matrix = apply_mask(matrix, high_mask, high_value)
    return matrix

def simplest_cb(img, percent):
    assert img.shape[2] == 3
    assert percent > 0 and percent < 100
    half_percent = percent / 200.0
    channels = cv2.split(img)
    out_channels = []
    for channel in channels:
        assert len(channel.shape) == 2
        # [EN] find the low and high precentile values (based on the input percentile)
        # [TR] düşük ve yüksek yüzdelik değerleri girdiye göre buluyoruz
        height, width = channel.shape
        vec_size = width * height
        flat = channel.reshape(vec_size)
        assert len(flat.shape) == 1
        flat = np.sort(flat)
        n_cols = flat.shape[0]
        low_val  = flat[math.floor(n_cols * half_percent)]
        high_val = flat[math.ceil( n_cols * (1.0 - half_percent))]
        # [EN] saturate below the low percentile and above the high percentile
        thresholded = apply_threshold(channel, low_val, high_val)
        # [EN] scale the channel
        normalized = cv2.normalize(thresholded, thresholded.copy(), 0, 255, cv2.NORM_MINMAX)
        out_channels.append(normalized)

    return cv2.merge(out_channels)



def clearing(img):
    factor = 0.6
    b,g,r = cv2.split(img)
    clahe_b = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(5,5))
    clahe_g = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(5,5))
    clahe_r = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(5,5))
    b=clahe_b.apply(b)
    g=clahe_g.apply(g)
    r=clahe_r.apply(r)
    img = cv2.merge([b,g,r])
    img = simplest_cb(img, 1.2)
    # gaussian_3 = cv2.GaussianBlur(img, (3, 3), 3.0)
    # img = cv2.addWeighted(img, 1.3,img, -0.5, 0, img)
    img = adjust_gamma(img, gamma=1.2)
    return img
    







In [28]:
for dirpath, dirnames, filenames in os.walk(test_dir):
    for x in filenames:
        if x[-3:] == 'jpg':
            try:
                img = cv2.imread(os.path.join(test_dir,x))
                img = clearing(img) 
                cv2.imshow('thresh', img)
                cv2.waitKey(0)
                if cv2.waitKey(0) & 0xFF == ord('q'):
                    break
            except:
                print(os.path.join(test_dir,x))   
    
       
cv2.destroyAllWindows()



In [31]:
for dirpath, dirnames, filenames in os.walk(test_dir):
        for x in filenames:
            if x[-3:] == 'jpg':
                try:
                    img = cv2.imread(os.path.join(test_dir,x))
                    img = clearing(img) 
                    cv2.imwrite(os.path.join(test_out_dir,x),img) 
                except:
                    print(os.path.join(test_dir,x))


In [43]:
for dirpath, dirnames, filenames in os.walk(test_dir):
        for x in filenames:
            if x[-3:] == 'jpg':
                    img = cv2.imread(os.path.join(test_dir,x))
                    img = clearing(img) 
                    cv2.imwrite(os.path.join(test_out_dir,x),img) 
                    # print(os.path.join(test_dir,x))